#### Define your project and region below. If you are not authenticated to GCP, do it by oncommenting the line below the definitions.

In [ ]:
PROJECT_ID = "SOME_PROJECT"
REGION = "YOUR_REGION" #though us-central is cheaper
PIPELINE_ROOT = "gs://SOME_BUCKET/SOME_FOLDER"
#!gcloud auth login

#### Imports

Our imports:

 * Artifact,
 * Dataset,
 * Input,
 * Model,
 * Output,
 * Metrics,
 * ClassificationMetrics,

Are powerful, metadata rich handles for objects "Artifacts", or its inherited classes. By using them, as shown below, we can manage paths, save and download them. The paths used are actually system path, as it is saved and shared between components via [GCS Fuse](https://cloud.google.com/storage/docs/gcs-fuse). 

`component` is a decorator used for transforming a function into a KFP component. It allows us, for example, to set dependencies and base images for each of our components, with a easy-to-use and simple API.

In [ ]:
from typing import NamedTuple

from kfp.v2 import dsl
from kfp.v2.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component)

from kfp.v2 import compiler

As from (GCP AI Platform Official Github)[https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/notebooks/unofficial/pipelines/lightweight_functions_component_io_kfp.ipynb], accessed in 2021-05-19:


KFP Python function-based components
A Kubeflow pipeline component is a self-contained set of code that performs one step in your ML workflow. A pipeline component is composed of:

The component code, which implements the logic needed to perform a step in your ML workflow.
A component specification, which defines the following:
 * The component’s metadata, its name and description.
 * The component’s interface, the component’s inputs and outputs.
 * The component’s implementation, the Docker container image to run, how to pass inputs to your component code, and how to get the component’s outputs.
 
Lightweight Python function-based components make it easier to iterate quickly by letting you build your component code as a Python function and generating the component specification for you. This notebook shows how to create Python function-based components for use in Vertex AI Pipelines.

Python function-based components use the Kubeflow Pipelines SDK to handle the complexity of passing inputs into your component and passing your function’s outputs back to your pipeline.

There are two categories of inputs/outputs supported in Python function-based components: artifacts and parameters.

* Parameters are passed to your component by value and typically contain int, float, bool, or small string values.
* Artifacts are passed to your component as a reference to a path, to which you can write a file or a subdirectory structure. In addition to the artifact’s data, you can also read and write the artifact’s metadata. This lets you record arbitrary key-value pairs for an artifact such as the accuracy of a trained model, and use metadata in downstream components – for example, you could use metadata to decide if a model is accurate enough to deploy for predictions.

#### Our use case

Is to create three components, that will let us create and save a dataset, train a model, and evaluate it, saving beautiful, meaningfull classification plots for them.

As you will see, our components have dependencies on `pandas`, `sklearn`, and `xgboost`.

We use `Output[Dataset]` (or Model, or ClassificationMetrics) objects to create unique filepaths to save objects during the component's execution. We can then access them as:
```python

some_op = component()
some_output_object = some_op.outputs["some_object_name"].
```

Below we create two `Output[Dataset]` objects to save the train and test split of our model. The next operators will receive some inputs and handle the previously saved files on their processing steps.

In [ ]:
@component(
    packages_to_install = [
        "pandas",
        "sklearn"
    ],
)
def get_data(
    dataset_train: Output[Dataset],
    dataset_test: Output[Dataset]
    
):
    
    from sklearn import datasets
    from sklearn.model_selection import train_test_split as tts
    import pandas as pd
    # import some data to play with
    
    data_raw = datasets.load_breast_cancer()
    data = pd.DataFrame(data_raw.data, columns=data_raw.feature_names)
    data["target"] = data_raw.target
    
    train, test = tts(data, test_size=0.3)
    
    train.to_csv(dataset_train.path)
    test.to_csv(dataset_test.path)

#### The training component

Will receive an `Input[Dataset]` object, will be used from the outputs of the `get_data()` operator. It outputs an `Output[Model]` object, which will have some metadata written about itself.

We will use the `Output[Model]` from component train and `Output[Dataset]` (the test one) from `get_data()` to evaluate the model.

In [ ]:
@component(
    packages_to_install = [
        "pandas",
        "sklearn",
        "xgboost"
    ],
)
def train_xgb_model(
    dataset: Input[Dataset],
    model_artifact: Output[Model]
):
    
    from xgboost import XGBClassifier
    import pandas as pd
    
    data = pd.read_csv(dataset.path)

    model = XGBClassifier(
        objective="binary:logistic"
    )
    model.fit(
        data.drop(columns=["target"]),
        data.target,
    )

    score = model.score(
        data.drop(columns=["target"]),
        data.target,
    )

    model_artifact.metadata["train_score"] = float(score)
    model_artifact.metadata["framework"] = "XGBoost"
    
    model.save_model(model_artifact.path)

#### To evaluate the model

We will receibe the inputs and create some specific outputs. `Output[ClassificationMetrics]` lets us create beautiful plots on the UI, and `Output[Metrics]` lets us log arbitrary metrics onto it. We will use `sklearn` for the metric gathering and then convert everything to list for Vertex AI runner to be able to plot. 

In [ ]:
@component(
    packages_to_install = [
        "pandas",
        "sklearn",
        "xgboost"
    ],
)
def eval_model(
    test_set: Input[Dataset],
    xgb_model: Input[Model],
    metrics: Output[ClassificationMetrics],
    smetrics: Output[Metrics]
):
    from xgboost import XGBClassifier
    import pandas as pd
    
    data = pd.read_csv(test_set.path)
    model = XGBClassifier()
    model.load_model(xgb_model.path)
    
    score = model.score(
        data.drop(columns=["target"]),
        data.target,
    )
    
    from sklearn.metrics import roc_curve
    y_scores =  model.predict_proba(data.drop(columns=["target"]))[:, 1]
    fpr, tpr, thresholds = roc_curve(
         y_true=data.target.to_numpy(), y_score=y_scores, pos_label=True
    )
    metrics.log_roc_curve(fpr.tolist(), tpr.tolist(), thresholds.tolist())
    
    from sklearn.metrics import confusion_matrix
    y_pred = model.predict(data.drop(columns=["target"]))
    
    metrics.log_confusion_matrix(
       ["False", "True"],
       confusion_matrix(
           data.target, y_pred
       ).tolist(),  # .tolist() to convert np array to list.
    )
    
    xgb_model.metadata["test_score"] = float(score)
    smetrics.log_metric("score", float(score))

#### The final step is to create the Pipeline

Notice that we get outputs from previous steps here. We then compile it into a `.json` file.

In [ ]:
@dsl.pipeline(
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root=PIPELINE_ROOT,
    # A name for the pipeline. Use to determine the pipeline Context.
    name="pipeline-test-1",
)
def pipeline():
    dataset_op = get_data()
    train_op = train_xgb_model(dataset_op.outputs["dataset_train"])
    eval_op = eval_model(
        test_set=dataset_op.outputs["dataset_test"],
        xgb_model=train_op.outputs["model_artifact"]
    )
    
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='xgb_pipe.json')

#### If you are authenticated to GCP and set everything up there

This snipped should create the run and a link for you to get to it.

Also, be sure your Vertex AI API is activated.

In [ ]:
from kfp.v2.google.client import AIPlatformClient

api_client = AIPlatformClient(
                project_id=PROJECT_ID,
                region=REGION
                )

response = api_client.create_run_from_job_spec(
    'xgb_pipe.json',
)